In [20]:
import yaml
import json
import base64
import hashlib
import wget
import zipfile
import os
import shutil

### LOGGING OUTLINE

In [4]:
def log(msg):
    print('[LAUNCHER] ' + msg.upper())

### LOAD YAML FILE

In [9]:
def load_yaml(path):
    with open(path, mode='r') as file:
        return yaml.load(file, Loader=yaml.FullLoader)

### LOAD & SAVE JSON FILE

In [10]:
def load_json(path):
    with open(path) as json_file:
        return json.load(json_file)

In [11]:
def save_json(data, path):
    with open(path, 'w') as outfile:
        json.dump(data, outfile)

### ENCODE & DECODE FOR BASE64

In [12]:
def encode(data):
    
    # STRINGIFY & CONVERT TO BYTES
    stringified = json.dumps(data)
    to_bytes = str.encode(stringified)
    
    # ENCODE
    encoded = base64.b64encode(to_bytes)
    
    # RETURN AS STRING
    return encoded.decode()

In [13]:
def decode(compressed):
    
    # ATTEMPT TO DECODE & PARSE AS JSON
    try:
        to_bytes = base64.b64decode(compressed)
        return json.loads(to_bytes)
    
    # OTHERWISE, RETURN EMPTY OBJECT
    except:
        return {}

### FILTER ZEROS FROM BACKLOG

In [14]:
def filter_backlog(data):
    
    # FILTER ZEROS
    filtered = filter(lambda x: x != '0x0000000000000000000000000000000000000000', data)
    
    # CONVERT TO LIST & RETURN
    return list(filtered)

### COMPARE DISCOVERY PARAMS

In [17]:
def compare_discovery(data, base):
    
    # RESULT CONTAINER
    result = []
    
    # LOOP THROUGH DATA KEYS
    for key in data:
        
        # IF THE KEY EXISTS IN THE BASE DICT
        if key in base:
            
            # IF THE VALUE IS SAME IN BOTH DATASET
            if data[key] == base[key]:
                result.append(True)
                
            # OTHERWISE, DEFAULT TO FALSE
            else:
                result.append(False)
                
        # OTHERWISE, DEFAULT TO FALSE
        else:
            result.append(False)
            
    # FINALLY RETURN RESULT
    return result

### CHECKSUM STUFF

In [2]:
def generate_checksum(path):
    with open(path, 'rb') as file:
        return hashlib.sha224(file.read()).hexdigest()

In [17]:
def verify_checksums(prefix):
    
    # LOAD CHECKSUMS & EXTRACT FILENAMES
    checksums = load_json(prefix + 'checksums.json')
    files = list(checksums.keys())
    
    # RESULT CONTAINER
    results = []
    
    # LOOP THROUGH FILES
    for file in files:
        
        # GENERATE CHECKSUM FOR FILE
        checksum = generate_checksum(prefix + file)
        
        # VERIFY & PUSH RESULT
        results.append(checksum == checksums[file])
    
    # IF ALL FILES PASS, RETURN TRUE
    if (results.count(False) == 0):
        return True
    
    # OTHERWISE, RETURN FALSE
    return False

### DOWNLOAD FILE -- RETURN FILENAME

In [8]:
def download(url):
    return wget.download(url)

### UNZIP DIRECTORY

In [7]:
def unzip(path, target):
    with zipfile.ZipFile(path, 'r') as zip_ref:
        zip_ref.extractall(target)

### CLEAN UP UNNECESSARY FILES & DIRS

In [19]:
def clear(downloaded_zip):
    
    # REMOVE THE DOWNLOADED ZIP FILE
    os.remove(downloaded_zip)
    
    # REMOVE THE TEMP DIR
    shutil.rmtree('temp')